## **Exploring Semantic Link**
Notebook with examples and explanations for basic functions using Semantic Link
<br><br>Note: This notebook is built to work with (and run in) the [Fabric Healthcare Demo workspace](https://github.com/isinghrana/fabric-samples-healthcare/tree/main/analytics-bi-directlake-starschema)
<br>Complete that install first, then import this notebook to that workspace.
<br><br> Make sure to update your Lakehouse! Select 'Lakehouses' on the left, 'Remove All Lakehouses', then add a Lakehouse you have access to or create a new one.
<br><br>[Documentation and tutorials](https://learn.microsoft.com/en-us/fabric/data-science/semantic-link-overview)
<br><br>[Detailed SemPy Function listing](https://learn.microsoft.com/en-us/python/api/semantic-link-sempy/sempy.fabric?view=semantic-link-python)

In [ ]:
# Import the SemanticLink package and other dependencies
# SemanticLink is included in the default environment starting the with the Spark3.4 env, no need to pip install any more
import sempy.fabric as fabric
import pandas as pd


In [ ]:
# The dir command will retrieve a list of functions available
dir(fabric)

In [ ]:
# There is a handy help function built-in as well. Use the syntax 'help(fabric.[function name from above])
help(fabric.model_memory_analyzer)

### List functions - gather information on semantic model elements
Using the 'List' functions you can quickly compile information about your semantic models.

In [ ]:
# fabric.list_capacities()
fabric.list_datasets()
# fabric.list_workspaces()

In [ ]:
# Specify the semantic model we're going to use for the rest of the notebook
sel_model = "cms_semantic_model"

In [ ]:
# List all tables in the specified data model, add the model name to the dataframe
tables_df = fabric.list_tables(sel_model,extended=True)
tables_df['SemanticModelName'] = sel_model

# List all columns in the specified data model, add the model name to the dataframe
columns_df = fabric.list_columns(sel_model,extended=True)
columns_df['SemanticModelName'] = sel_model

# List all measures in the specified data model, add the model name to the dataframe
measures_df = fabric.list_measures(sel_model)
measures_df['SemanticModelName'] = sel_model


In [ ]:
# tables_df
# columns_df
measures_df

In [ ]:
# Prep for writing back to LH table - remove spaces in column names
tables_df.columns = tables_df.columns.str.replace(' ','')
columns_df.columns = columns_df.columns.str.replace(' ','')
measures_df.columns = measures_df.columns.str.replace(' ','')

In [ ]:
# Review the final cleaned up dataframes
# tables_df
# columns_df
measures_df

#### All this can be easily written back to tables in the LH to build out a semantic model/data dictionary as documentation:

In [ ]:
# I've had the best luck converting the pandas df to a spark df before writing to the LH. Using overwrite in this scenario, but you could also append (with a timestamp)
spark.createDataFrame(tables_df).write.mode('overwrite').option("mergeSchema", "true").saveAsTable("ModelTables")
spark.createDataFrame(columns_df).write.mode('overwrite').option("mergeSchema", "true").saveAsTable("ModelColumns")
spark.createDataFrame(measures_df).write.mode('overwrite').option("mergeSchema", "true").saveAsTable("ModelMeasures")

In [ ]:
# You can also get all of the above in one line of code (essentially Vertipaq analyzer)
fabric.model_memory_analyzer(sel_model)

#### We can also take a look at relationship information

In [ ]:
from sempy.relationships import plot_relationship_metadata
from sempy.relationships import find_relationships
from sempy.fabric import list_relationship_violations
from sempy.dependencies import plot_dependency_metadata

In [ ]:
help(find_relationships)

In [ ]:
# List all relationships
relationships_df = fabric.list_relationships(sel_model)
relationships_df

In [ ]:
# Visually plot relationships
plot_relationship_metadata(relationships_df)

### Extract data from the model
There are several ways to access the model data, either using the business logic built-in via measures or DAX, or by querying the model directly

In [ ]:
# Evaluate a measure from a dataset. Dataset, Measure, GroupBy column(s)
drugcosts_msr_df = (
    fabric.evaluate_measure(
        sel_model,
        "Total Drug Cost",
        groupby_columns=["cms_provider_drug_costs_star[Year]"]
        )
)
drugcosts_msr_df = drugcosts_msr_df.sort_values(["Total Drug Cost"],ascending=False)

display(drugcosts_msr_df)

In [ ]:
# Evaluate DAX directly
dax_example_df = fabric.evaluate_dax(dataset=sel_model,
    dax_string=
    """
    EVALUATE 
    SUMMARIZECOLUMNS(
        'cms_provider_drug_costs_star'[Year],
        "Total Drug Cost", 'cms_provider_drug_costs_star'[Total Drug Cost]
    )"""
)
dax_example_df = dax_example_df.sort_values(["[Total Drug Cost]"],ascending=False)
display(dax_example_df)

In [ ]:
# Direct Query an attached Lakehouse (Update the Lakehouse in the table signifiers)
from pyspark.sql.functions import month, year, col, desc, asc
costs_df = spark.sql("SELECT * FROM cms_lakehouse.cms_provider_drug_costs")
# display(costs_df)

# Aggregate costs to a yearly level
costs_agg_df = costs_df.withColumn("BrandName",costs_df.Brnd_Name)\
.groupBy("BrandName")\
.sum("Tot_Drug_Cst")\
# .orderBy("Tot_Drug_Cst")
costs_agg_df.sort(desc("sum(Tot_Drug_Cst)")).show()
# display(costs_agg_df)

### Access the REST APIs quickly and easily
No need to worry about authentication, scopes, etc. A user running a notebook can access whatever endpoints are available based on their role permissions
<br><br>[Power BI REST API Docs](https://learn.microsoft.com/en-us/rest/api/power-bi/)
<br><br>[Fabric REST API Docs](https://learn.microsoft.com/en-us/rest/api/fabric/articles/)

In [ ]:
import json 

# Initialize the PowerBI REST client
client = fabric.PowerBIRestClient()

In [ ]:
# Make a 'GET' request to the selected endpoint. Notebook user must have permissions to make API request
# ATTN: You'll need to update the workspace id after ....groups/[XXXXX]/datasets'
response = client.get('v1.0/myorg/groups/173d7d53-7ab2-4fc6-b930-31d6b9db01f9/datasets')
response

In [ ]:
# Parse the response and normalize to a dataframe
data = json.loads(response.text)
datasets_df = pd.json_normalize(data['value'])
display(datasets_df)

In [ ]:
# Initialize the Fabric REST Client
client_fab = fabric.FabricRestClient()

In [ ]:
# Make a 'GET' request to the selected endpoint. Notebook user must have permissions to make API request
# ATTN: You'll need to update the workspace id after ....groups/[XXXXX]/datasets'
response = client_fab.get('v1/workspaces/173d7d53-7ab2-4fc6-b930-31d6b9db01f9/items')
response

In [ ]:
# Parse the response and normalize to a dataframe
data = json.loads(response.text)
fabitems_df = pd.json_normalize(data['value'])
display(fabitems_df)

### You can also use other Python libraries
Such as powerbiclient to show a report in your notebook

In [ ]:
# Use the PowerBIRESTClient to display a report
reports_df = fabric.list_reports()
# reports_df # Show the list of reports
report_name = "cms_report"
report_id = reports_df[reports_df['Name'] == report_name]['Id'].iloc[0]
# report_id # show the report id selected

In [ ]:
from powerbiclient import Report
report = Report(group_id=None, report_id=report_id)
report